### Задача:

Проанализировать зависимость оплат от прохождения обучения. Для этого нужно определить существует ли различие в частоте и средней величине оплат между 3 группами пользователей:
- Пользователями, которые прошли обучение хотя бы раз.
- Пользователями, которые начали обучение, но не прошли его ни разу.
- Пользователями, которые не начинали обучение, а сразу же перешли к выбору уровня сложности.

Существует ли разница во времени между пользователями с разным уровнем сложности и их первой оплатой?

In [1]:
import pandas as pd
import psycopg2
import psycopg2.extras 
import numpy as np

In [2]:
def getEventsData():
    query = '''SELECT e.* FROM case8.events e
    WHERE e.user_id in
    (SELECT DISTINCT(user_id) 
    FROM case8.events 
    WHERE event_type = 'registration' 
    AND start_time >= '2017-01-01'
    AND start_time < '2018-01-01')
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

def getPurchaseData():
    query = '''SELECT p.* FROM case8.purchase p
    WHERE p.user_id in
    (SELECT DISTINCT(user_id) 
    FROM case8.events 
    WHERE event_type = 'registration' 
    AND start_time >= '2017-01-01'
    AND start_time < '2018-01-01')
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

events_df = pd.DataFrame(getEventsData())
purchase_df = pd.DataFrame(getPurchaseData())

In [3]:
events_df = events_df[events_df['event_type'] == 'level_choice']
# удалим столбец tutorial_id
del events_df['tutorial_id']
events_df.head()

,event_type,selected_level,start_time,user_id,id
19,level_choice,medium,2017-01-01 20:37:22,27835,80327
23,level_choice,hard,2017-01-01 22:37:50,27839,80331
36,level_choice,medium,2017-01-02 05:18:42,27840,80344
38,level_choice,hard,2017-01-02 06:19:18,27845,80346
45,level_choice,easy,2017-01-02 08:46:03,27842,80353


In [4]:
events_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8342 entries, 19 to 66957
Data columns (total 5 columns):
event_type        8342 non-null object
selected_level    8342 non-null object
start_time        8342 non-null datetime64[ns]
user_id           8342 non-null int64
id                8342 non-null int64
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 391.0+ KB


In [5]:
events_df['selected_level'].unique()

array(['medium', 'hard', 'easy'], dtype=object)

In [6]:
events_df = events_df.rename(columns={'id':'event_id'})
purchase_df = purchase_df.rename(columns={'id':'purchase_id', 'start_time':'purchase_start_time'})
purchase_df.head()

,user_id,purchase_start_time,amount,purchase_id
0,37878,2017-06-30 17:05:21,150,17668
1,47216,2017-12-22 06:30:31,25,18396
2,35532,2017-05-21 04:23:32,150,17475
3,42583,2017-10-17 13:52:14,100,18027
4,32321,2017-03-20 06:51:27,150,17202


#### Пользователи, выбравшие "easy" уровень

In [7]:
users_easy_df = events_df[events_df['selected_level'] == 'easy']

In [8]:
users_easy_count = users_easy_df['user_id'].nunique()
print('Количество пользователей: ' + str(users_easy_count))

Количество пользователей: 2448


In [9]:
users_easy_df = users_easy_df.merge(purchase_df, how = 'inner', on = 'user_id')
users_easy_df['timedelta'] = users_easy_df['purchase_start_time'] - users_easy_df['start_time']
users_easy_df.head()

,event_type,selected_level,start_time,user_id,event_id,purchase_start_time,amount,purchase_id,timedelta
0,level_choice,easy,2017-01-04 16:18:39,27884,80488,2017-01-08 19:37:34,150,16854,4 days 03:18:55
1,level_choice,easy,2017-01-09 21:34:23,28090,81064,2017-01-15 23:42:55,100,16873,6 days 02:08:32
2,level_choice,easy,2017-01-11 18:44:45,28182,81472,2017-01-12 02:46:01,200,16865,0 days 08:01:16
3,level_choice,easy,2017-01-11 21:10:51,28207,81513,2017-01-12 21:00:24,150,16867,0 days 23:49:33
4,level_choice,easy,2017-01-12 16:48:24,28254,81660,2017-01-19 22:08:40,50,16892,7 days 05:20:16


In [10]:
users_easy_purchase_count = users_easy_df[users_easy_df['amount'] > 0]['user_id'].nunique()
percent_purchase_easy_users = users_easy_purchase_count / users_easy_count
print('Процент пользователей, оплативших тренировки: {:.2%}'.format(percent_purchase_easy_users))
print('Средний чек: {:.5}'.format(users_easy_df['amount'].mean()))
print('Среднее время между выбором сложности и оплатой: ' + str(users_easy_df['timedelta'].mean()))

Процент пользователей, оплативших тренировки: 7.72%
Средний чек: 114.95
Среднее время между выбором сложности и оплатой: 3 days 14:58:52.941798


#### Пользователи, выбравшие "medium" уровень

In [11]:
users_medium_df = events_df[events_df['selected_level'] == 'medium']

In [12]:
users_medium_count = users_medium_df['user_id'].nunique()
print('Количество пользователей: ' + str(users_medium_count))

Количество пользователей: 4645


In [13]:
users_medium_df = users_medium_df.merge(purchase_df, how = 'inner', on = 'user_id')
users_medium_df['timedelta'] = users_medium_df['purchase_start_time'] - users_medium_df['start_time']
users_medium_df.head()

,event_type,selected_level,start_time,user_id,event_id,purchase_start_time,amount,purchase_id,timedelta
0,level_choice,medium,2017-01-07 05:29:30,27973,80745,2017-01-13 21:50:00,150,16869,6 days 16:20:30
1,level_choice,medium,2017-01-07 10:46:14,27981,80761,2017-01-07 23:20:25,50,16852,0 days 12:34:11
2,level_choice,medium,2017-01-08 00:00:52,28010,80840,2017-01-10 05:32:47,50,16858,2 days 05:31:55
3,level_choice,medium,2017-01-08 14:47:35,28020,80885,2017-01-11 21:43:03,50,16863,3 days 06:55:28
4,level_choice,medium,2017-01-08 17:06:39,28033,80900,2017-01-16 05:08:41,100,16874,7 days 12:02:02


In [14]:
users_medium_purchase_count = users_medium_df[users_medium_df['amount'] > 0]['user_id'].nunique()
percent_purchase_medium_users = users_medium_purchase_count / users_medium_count
print('Процент пользователей, оплативших тренировки: {:.2%}'.format(percent_purchase_medium_users))
print('Средний чек: {:.5}'.format(users_medium_df['amount'].mean()))
print('Среднее время между выбором сложности и оплатой: ' + str(users_medium_df['timedelta'].mean()))

Процент пользователей, оплативших тренировки: 20.86%
Средний чек: 109.52
Среднее время между выбором сложности и оплатой: 3 days 23:14:13.165118


#### Пользователи, выбравшие "hard" уровень

In [15]:
users_hard_df = events_df[events_df['selected_level'] == 'hard']

In [16]:
users_hard_count = users_hard_df['user_id'].nunique()
print('Количество пользователей: ' + str(users_hard_count))

Количество пользователей: 1249


In [17]:
users_hard_df = users_hard_df.merge(purchase_df, how = 'inner', on = 'user_id')
users_hard_df['timedelta'] = users_hard_df['purchase_start_time'] - users_hard_df['start_time']
users_hard_df.head()

,event_type,selected_level,start_time,user_id,event_id,purchase_start_time,amount,purchase_id,timedelta
0,level_choice,hard,2017-01-02 06:19:18,27845,80346,2017-01-03 18:53:43,100,16845,1 days 12:34:25
1,level_choice,hard,2017-01-04 05:56:32,27865,80459,2017-01-04 14:46:10,250,16846,0 days 08:49:38
2,level_choice,hard,2017-01-05 11:59:50,27910,80545,2017-01-07 12:11:34,100,16849,2 days 00:11:44
3,level_choice,hard,2017-01-05 17:39:02,27911,80572,2017-01-07 08:19:12,50,16848,1 days 14:40:10
4,level_choice,hard,2017-01-06 00:32:47,27940,80620,2017-01-07 13:16:41,200,16850,1 days 12:43:54


In [18]:
users_hard_purchase_count = users_hard_df[users_hard_df['amount'] > 0]['user_id'].nunique()
percent_purchase_hard_users = users_hard_purchase_count / users_hard_count
print('Процент пользователей, оплативших тренировки: {:.2%}'.format(percent_purchase_hard_users))
print('Средний чек: {:.5}'.format(users_hard_df['amount'].mean()))
print('Среднее время между выбором сложности и оплатой: ' + str(users_hard_df['timedelta'].mean()))

Процент пользователей, оплативших тренировки: 35.39%
Средний чек: 111.6
Среднее время между выбором сложности и оплатой: 3 days 07:20:41.420814


### Анализ групп

#### Пользователи, выбравшие "easy" уровень

In [19]:
print('Процент пользователей, оплативших тренировки: {:.2%}'.format(percent_purchase_easy_users))
print('Средний чек: {:.5}'.format(users_easy_df['amount'].mean()))
print('Среднее время между выбором сложности и оплатой: ' + str(users_easy_df['timedelta'].mean()))

Процент пользователей, оплативших тренировки: 7.72%
Средний чек: 114.95
Среднее время между выбором сложности и оплатой: 3 days 14:58:52.941798


#### Пользователи, выбравшие "medium" уровень

In [20]:
print('Процент пользователей, оплативших тренировки: {:.2%}'.format(percent_purchase_medium_users))
print('Средний чек: {:.5}'.format(users_medium_df['amount'].mean()))
print('Среднее время между выбором сложности и оплатой: ' + str(users_medium_df['timedelta'].mean()))

Процент пользователей, оплативших тренировки: 20.86%
Средний чек: 109.52
Среднее время между выбором сложности и оплатой: 3 days 23:14:13.165118


#### Пользователи, выбравшие "hard" уровень

In [21]:
print('Процент пользователей, оплативших тренировки: {:.2%}'.format(percent_purchase_hard_users))
print('Средний чек: {:.5}'.format(users_hard_df['amount'].mean()))
print('Среднее время между выбором сложности и оплатой: ' + str(users_hard_df['timedelta'].mean()))

Процент пользователей, оплативших тренировки: 35.39%
Средний чек: 111.6
Среднее время между выбором сложности и оплатой: 3 days 07:20:41.420814


### Выводы:
1. Имеется зависимость между процентами оплаты по пользователям, выбравшим соответствующий уровень тренировок: чем выше сложность, тем выше процент оплаты, возможно, это связано с большей заинтересованностью продвинутого пользователя, а не с низким качеством материала тренировок уровня "easy".
2. Средний чек колеблется между группами незначительно.
3. Среднее время между выбором сложности и оплатой также колеблется незначительно, составляет ~ 3 дня.